# Notebook 2: Isolation Forest para Detecção de Fraudes

## Objetivo
1.  Entender o algoritmo **Isolation Forest (iForest)**.
2.  Por que ele é eficaz para grandes volumes de dados (auditoria em massa).
3.  Detectar anomalias em um dataset de transações financeiras simuladas com múltiplas dimensões.

## O Conceito
O iForest funciona isolando observações. A lógica é: anomalias são **poucas** e **diferentes**. 
Se construirmos árvores de decisão aleatórias, é muito mais fácil (requer menos cortes) isolar um ponto anômalo do que um ponto normal (que está aglomerado com muitos outros).

- **Menos cortes (caminho curto)** = Alta probabilidade de anomalia.
- **Mais cortes (caminho longo)** = Dado normal.

In [ ]:
!pip install -q pyod pandas matplotlib seaborn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyod.models.iforest import IForest
from pyod.utils.data import generate_data

plt.rcParams['figure.figsize'] = (10, 6)

## 1. Gerando Dados Multidimensionais
Ao contrário do KNN, o iForest escala bem com mais dimensões. Vamos simular dados com 5 características (features), como se fossem:
1.  Valor da Transação
2.  Hora do dia
3.  Idade da conta
4.  Frequência de uso prévio
5.  Risco do MCC (Merchant Category Code)

In [ ]:
n_train = 1000
n_test = 500
contamination = 0.03 # 3% de fraudes

# Gerando dados com 5 features
X_train, X_test, y_train, y_test = generate_data(
    n_train=n_train, 
    n_test=n_test, 
    n_features=5, 
    contamination=contamination, 
    random_state=123
)

# Visualizando as primeiras linhas do treino
feature_names = ['Valor', 'Hora', 'Idade_Conta', 'Freq_Uso', 'Risco_MCC']
df_train = pd.DataFrame(X_train, columns=feature_names)
df_train.head()

## 2. Treinando o Isolation Forest
O iForest é um 'ensemble' method (usa várias árvores).

In [ ]:
# Inicializa o modelo
# n_estimators: número de árvores (padrao 100)
# max_samples: amostras por árvore (auto)
clf_if = IForest(n_estimators=100, contamination=contamination, random_state=42, n_jobs=-1)

# Treina
clf_if.fit(X_train)

# Obtendo scores
y_train_scores = clf_if.decision_scores_
y_train_pred = clf_if.labels_

## 3. Análise da Distribuição dos Scores
Uma forma excelente de definir um 'corte' (threshold) em auditoria é olhar o histograma dos scores de anomalia.

In [ ]:
plt.hist(y_train_scores, bins=50, color='skyblue', edgecolor='black')
plt.title('Histograma dos Scores de Anomalia (Isolation Forest)')
plt.xlabel('Score de Anomalia (Quanto maior, mais suspeito)')
plt.ylabel('Frequência')
plt.axvline(clf_if.threshold_, color='red', linestyle='dashed', linewidth=2, label='Threshold Automático')
plt.legend()
plt.show()

Perceba como a maioria dos dados tem score baixo (lado esquerdo) e uma cauda pequena à direita (os outliers).

## 4. Visualização 2D (usando PCA)
Como temos 5 dimensões, não conseguimos plotar tudo. Vamos usar PCA (Principal Component Analysis) apenas para reduzir para 2D e visualizar o resultado do iForest.

In [ ]:
from sklearn.decomposition import PCA

# Reduzindo para 2 componentes principais
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)

plt.figure(figsize=(10, 6))
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train_pred, cmap='coolwarm', edgecolor='k', s=40)
plt.title('Visualização PCA (2D) da Classificação do iForest')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.show()

Nota: Os pontos vermelhos (outliers) geralmente estão nas periferias dos clusters principais.

## Conclusão
Isolation Forest é rápido e eficaz. É o algoritmo 'go-to' (padrão) para começar análises de anomalia em grandes datasets tabulares.

**Próximo:** LOF (Local Outlier Factor) para detectar anomalias que o iForest pode perder (outliers locais).